In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# from AST425 import *

2025-05-08 09:02:44.856386: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-08 09:02:44.857413: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 09:02:44.861405: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-08 09:02:44.873357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746709364.892921   19175 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746709364.89

In [2]:
#shape of VAE is 38, 100, 100, 2, 100, 100 38

In [3]:
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.labelsize'] = 13
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams['legend.fontsize'] = 13

Data preprocessing (same step for all methods):

In [4]:
data = pd.read_csv('../tsne_umap_tutorials/data/APOGEEDR17_GAIAEDR3_noflagfilter.csv', delimiter=',')

# data = pd.read_csv('APOGEEDR17_GAIAEDR3_noflagfilter.csv', delimiter=',')

In [5]:
# Spatial:
ra = data["RA"]   #APOGEE
dec = data["DEC"]   #APOGEE

# Identification:
apogee_ID = data["# APOGEE_ID_"]   #APOGEE
gaia_ID = data["GAIAEDR3_SOURCE_ID"]  # Gaia

# Kinematic:
parallax = data["GAIAEDR3_PARALLAX"]  # Gaia
pmra = data["GAIAEDR3_PMRA"]  # Gaia
pmra_err = data["GAIAEDR3_PMRA_ERROR"]  # Gaia
pmdec = data["GAIAEDR3_PMDEC"]  # Gaia
pmdec_err = data["GAIAEDR3_PMDEC_ERROR"]  # Gaia
RV = data["VHELIO_AVG"]   #APOGEE
RV_err = data["VERR"]   #APOGEE
#dist = data["dist"]   #APOGEE no dist in this dataset?
#dist_err = data["dist_err"]   #APOGEE
jr = data["jr"]   #APOGEE
jr_err = data["jr_err"]   #APOGEE
jz = data["jz"]   #APOGEE
jz_err = data["jz_err"]   #APOGEE
#jphi = data["jphi"]   #APOGEE no jphi in this dataset?
#jphi_err = data["jphi_err"]    #APOGEE

# # Spectral (useful for filtering):
TEFF_ERR = data["TEFF_ERR"]   #APOGEE
TEFF = data["TEFF"]   #APOGEE
LOGG_ERR = data["LOGG_ERR"]   #APOGEE
LOGG = data["LOGG"]   #APOGEE
SNR = data["SNR"]   #APOGEE
ASPCAPFLAG = data["ASPCAPFLAG"]
STARFLAG = data["STARFLAG"]

# # Chemical abundances from astroNN:
FE_H = data['FE_H'] 
C_FE = data['C_FE']
CI_FE = data['CI_FE']
N_FE = data['N_FE']
O_FE = data['O_FE']
MG_FE = data['MG_FE']
AL_FE = data['AL_FE']
SI_FE = data['SI_FE']
P_FE = data['P_FE']
S_FE = data['S_FE']
K_FE = data['K_FE']
CA_FE = data['CA_FE']
TI_FE = data['TI_FE']
TIII_FE = data['TIII_FE']
V_FE = data['V_FE']
CR_FE = data['CR_FE']
MN_FE = data['MN_FE']
CO_FE = data['CO_FE']
NI_FE = data['NI_FE']

# # Chemical abundance errors from astroNN:
FE_H_err = data["FE_H_ERR"] 
C_FE_err = data['C_FE_ERR']
CI_FE_err = data['CI_FE_ERR']
N_FE_err = data['N_FE_ERR']
O_FE_err = data['O_FE_ERR']
MG_FE_err = data['MG_FE_ERR']
AL_FE_err = data['AL_FE_ERR']
SI_FE_err = data['SI_FE_ERR']
P_FE_err = data['P_FE_ERR']
S_FE_err = data['S_FE_ERR']
K_FE_err = data['K_FE_ERR']
CA_FE_err = data['CA_FE_ERR']
TI_FE_err = data['TI_FE_ERR']
TIII_FE_err = data['TIII_FE_ERR']
V_FE_err = data['V_FE_ERR']
CR_FE_err = data['CR_FE_ERR']
MN_FE_err = data['MN_FE_ERR']
CO_FE_err = data['CO_FE_ERR']
NI_FE_err = data['NI_FE_ERR']

# # Number of stars in the initial sample of APOGEE DR16: 
print("There are {} stars in our initial sample".format(len(ra)))

cols = [ra, dec, apogee_ID, gaia_ID, parallax, pmra, pmra_err, pmdec, pmdec_err, RV, RV_err, #dist, dist_err,
         jr, jr_err,# jphi, jphi_err,
        jz, jz_err, TEFF, TEFF_ERR, LOGG, LOGG_ERR, SNR, ASPCAPFLAG, STARFLAG, FE_H, FE_H_err, C_FE, 
         C_FE_err, CI_FE, CI_FE_err, N_FE, N_FE_err, O_FE, O_FE_err, MG_FE, MG_FE_err, AL_FE, AL_FE_err, SI_FE,
         SI_FE_err, P_FE, P_FE_err, S_FE, S_FE_err, K_FE, K_FE_err, CA_FE, CA_FE_err, TI_FE, TI_FE_err, TIII_FE,
         TIII_FE_err, V_FE, V_FE_err, CR_FE, CR_FE_err, MN_FE, MN_FE_err, CO_FE, CO_FE_err, NI_FE, NI_FE_err]

There are 370060 stars in our initial sample


In [6]:
aspcapflags_filter = np.array(cols[20])==0
starflags_filter = np.array(cols[21])==0

filters = aspcapflags_filter*starflags_filter


filtered_data = []
for c in cols:
    a = np.array(c)[filters]
    filtered_data.append(a)
    
print("There are {} stars in our filtered sample".format(len(filtered_data[0])))

There are 144767 stars in our filtered sample


In [7]:
FE_H_filtered, C_FE_filtered, CI_FE_filtered = filtered_data[22], filtered_data[24], filtered_data[26]
N_FE_filtered, O_FE_filtered, MG_FE_filtered = filtered_data[28], filtered_data[30], filtered_data[32]
AL_FE_filtered, SI_FE_filtered, P_FE_filtered = filtered_data[34], filtered_data[36], filtered_data[38]
S_FE_filtered, K_FE_filtered, CA_FE_filtered = filtered_data[40], filtered_data[42], filtered_data[44]
TI_FE_filtered, TIII_FE_filtered, V_FE_filtered = filtered_data[46], filtered_data[48], filtered_data[50]
CR_FE_filtered, MN_FE_filtered, CO_FE_filtered, NI_FE_filtered = filtered_data[52], filtered_data[54], filtered_data[56], filtered_data[58]
RV_filtered  = filtered_data[9]

In [8]:
FE_H_err, C_FE_err, CI_FE_err = filtered_data[23], filtered_data[25], filtered_data[27]
N_FE_err, O_FE_err, MG_FE_err = filtered_data[29], filtered_data[31], filtered_data[33]
AL_FE_err, SI_FE_err, P_FE_err = filtered_data[35], filtered_data[37], filtered_data[39]
S_FE_err, K_FE_err, CA_FE_err = filtered_data[41], filtered_data[43], filtered_data[45]
TI_FE_err, TIII_FE_err, V_FE_err = filtered_data[47], filtered_data[49], filtered_data[51]
CR_FE_err, MN_FE_err, CO_FE_err, NI_FE_err = filtered_data[53], filtered_data[55], filtered_data[57], filtered_data[59]
RV_err  = filtered_data[10]

In [9]:
train_size = round(0.8 * len(filtered_data[0]))
test_size = round(0.1 * len(filtered_data[0]))
validation_size = round(0.1 * len(filtered_data[0]))

training_labels_raw = np.transpose(np.array([FE_H_filtered[:train_size], C_FE_filtered[:train_size], CI_FE_filtered[:train_size], 
                                        N_FE_filtered[:train_size], O_FE_filtered[:train_size], MG_FE_filtered[:train_size],
                                        AL_FE_filtered[:train_size], SI_FE_filtered[:train_size], P_FE_filtered[:train_size],
                                        S_FE_filtered[:train_size], K_FE_filtered[:train_size], CA_FE_filtered[:train_size],
                                        TI_FE_filtered[:train_size], TIII_FE_filtered[:train_size], V_FE_filtered[:train_size], 
                                        CR_FE_filtered[:train_size], MN_FE_filtered[:train_size], CO_FE_filtered[:train_size], NI_FE_filtered[:train_size]]))

error_training_labels_raw = np.transpose(np.array([FE_H_err[:train_size], C_FE_err[:train_size], CI_FE_err[:train_size], 
                                        N_FE_err[:train_size], O_FE_err[:train_size], MG_FE_err[:train_size],
                                        AL_FE_err[:train_size], SI_FE_err[:train_size], P_FE_err[:train_size],
                                        S_FE_err[:train_size], K_FE_err[:train_size], CA_FE_err[:train_size],
                                        TI_FE_err[:train_size], TIII_FE_err[:train_size], V_FE_err[:train_size], 
                                        CR_FE_err[:train_size], MN_FE_err[:train_size], CO_FE_err[:train_size], NI_FE_err[:train_size]]))


test_labels_raw = np.transpose(np.array([FE_H_filtered[train_size: train_size + test_size], C_FE_filtered[train_size: train_size + test_size], 
                                     CI_FE_filtered[train_size: train_size + test_size], N_FE_filtered[train_size: train_size + test_size],
                                     O_FE_filtered[train_size: train_size + test_size], MG_FE_filtered[train_size: train_size + test_size],
                                        AL_FE_filtered[train_size: train_size + test_size], SI_FE_filtered[train_size: train_size + test_size], 
                                     P_FE_filtered[train_size: train_size + test_size], S_FE_filtered[train_size: train_size + test_size], 
                                     K_FE_filtered[train_size: train_size + test_size], CA_FE_filtered[train_size: train_size + test_size],
                                        TI_FE_filtered[train_size: train_size + test_size], TIII_FE_filtered[train_size: train_size + test_size], 
                                     V_FE_filtered[train_size: train_size + test_size], CR_FE_filtered[train_size: train_size + test_size], 
                                     MN_FE_filtered[train_size: train_size + test_size], CO_FE_filtered[train_size: train_size + test_size], 
                                     NI_FE_filtered[train_size: train_size + test_size]]))
                                     
                                     
error_test_labels_raw = np.transpose(np.array([FE_H_err[train_size: train_size + test_size], C_FE_err[train_size: train_size + test_size], CI_FE_err[train_size: train_size + test_size], 
                                        N_FE_err[train_size: train_size + test_size], O_FE_err[train_size: train_size + test_size], MG_FE_err[train_size: train_size + test_size],
                                        AL_FE_err[train_size: train_size + test_size], SI_FE_err[train_size: train_size + test_size], P_FE_err[train_size: train_size + test_size],
                                        S_FE_err[train_size: train_size + test_size], K_FE_err[train_size: train_size + test_size], CA_FE_err[train_size: train_size + test_size],
                                        TI_FE_err[train_size: train_size + test_size], TIII_FE_err[train_size: train_size + test_size], V_FE_err[train_size: train_size + test_size], 
                                        CR_FE_err[train_size: train_size + test_size], MN_FE_err[train_size: train_size + test_size], CO_FE_err[train_size: train_size + test_size], NI_FE_err[train_size: train_size + test_size]]))


validation_labels_raw = np.transpose(np.array([FE_H_filtered[train_size + test_size: train_size + test_size + validation_size], C_FE_filtered[train_size + test_size: train_size + test_size + validation_size], 
                                     CI_FE_filtered[train_size + test_size: train_size + test_size + validation_size], N_FE_filtered[train_size + test_size: train_size + test_size + validation_size],
                                     O_FE_filtered[train_size + test_size: train_size + test_size + validation_size], MG_FE_filtered[train_size + test_size: train_size + test_size + validation_size],
                                        AL_FE_filtered[train_size + test_size: train_size + test_size + validation_size], SI_FE_filtered[train_size + test_size: train_size + test_size + validation_size], 
                                     P_FE_filtered[train_size + test_size: train_size + test_size + validation_size], S_FE_filtered[train_size + test_size: train_size + test_size + validation_size], 
                                     K_FE_filtered[train_size + test_size: train_size + test_size + validation_size], CA_FE_filtered[train_size + test_size: train_size + test_size + validation_size],
                                        TI_FE_filtered[train_size + test_size: train_size + test_size + validation_size], TIII_FE_filtered[train_size + test_size: train_size + test_size + validation_size], 
                                     V_FE_filtered[train_size + test_size: train_size + test_size + validation_size], CR_FE_filtered[train_size + test_size: train_size + test_size + validation_size], 
                                     MN_FE_filtered[train_size + test_size: train_size + test_size + validation_size], CO_FE_filtered[train_size + test_size: train_size + test_size + validation_size], 
                                     NI_FE_filtered[train_size + test_size: train_size + test_size + validation_size]]))
                                     
                                     
error_validation_labels_raw = np.transpose(np.array([FE_H_err[train_size + test_size: train_size + test_size + validation_size], C_FE_err[train_size + test_size: train_size + test_size + validation_size], CI_FE_err[train_size + test_size: train_size + test_size + validation_size], 
                                        N_FE_err[train_size + test_size: train_size + test_size + validation_size], O_FE_err[train_size + test_size: train_size + test_size + validation_size], MG_FE_err[train_size + test_size: train_size + test_size + validation_size],
                                        AL_FE_err[train_size + test_size: train_size + test_size + validation_size], SI_FE_err[train_size + test_size: train_size + test_size + validation_size], P_FE_err[train_size + test_size: train_size + test_size + validation_size],
                                        S_FE_err[train_size + test_size: train_size + test_size + validation_size], K_FE_err[train_size + test_size: train_size + test_size + validation_size], CA_FE_err[train_size + test_size: train_size + test_size + validation_size],
                                        TI_FE_err[train_size + test_size: train_size + test_size + validation_size], TIII_FE_err[train_size + test_size: train_size + test_size + validation_size], V_FE_err[train_size + test_size: train_size + test_size + validation_size], 
                                        CR_FE_err[train_size + test_size: train_size + test_size + validation_size], MN_FE_err[train_size + test_size: train_size + test_size + validation_size], CO_FE_err[train_size + test_size: train_size + test_size + validation_size], NI_FE_err[train_size + test_size: train_size + test_size + validation_size]]))


In [10]:
full_training_labels_raw = np.c_[training_labels_raw, error_training_labels_raw]
full_test_labels_raw = np.c_[test_labels_raw, error_test_labels_raw]
full_validation_labels_raw = np.c_[validation_labels_raw, error_validation_labels_raw]

In [11]:
full_training_labels_raw.shape, full_test_labels_raw.shape, full_validation_labels_raw.shape

((115814, 38), (14477, 38), (14476, 38))

In [12]:
np.min(full_training_labels_raw.T, axis=1), np.max(full_training_labels_raw.T, axis=1)

(array([-1.951497  , -2.1234312 , -2.119044  , -0.18978369, -0.7124527 ,
        -0.15558027, -0.47890055, -0.15707995, -0.97891414, -0.6047195 ,
        -2.2741165 , -0.40278423, -1.2907453 , -0.6157368 , -1.1658391 ,
        -1.391077  , -0.64225805, -3.6896837 , -0.2965148 ,  0.01358107,
         0.01762647,  0.02378747,  0.01708818,  0.01984493,  0.02071843,
         0.02232191,  0.01871411,  0.05818468,  0.01897749,  0.03516773,
         0.02518716,  0.02486172,  0.02255343,  0.0375476 ,  0.02254345,
         0.02762588,  0.01909786,  0.02221127]),
 array([5.7460123e-01, 8.3517190e-01, 5.6674075e-01, 3.6616602e+00,
        5.5585840e-01, 5.1025960e-01, 4.8934165e-01, 4.5210746e-01,
        7.5247210e+00, 9.0582700e-01, 1.0242985e+00, 2.7282690e-01,
        5.9501344e-01, 8.7636330e-01, 8.6697330e-01, 3.3210137e-01,
        3.5304812e-01, 5.6587050e-01, 2.1270613e-01, 4.5790780e-01,
        1.2761971e+00, 1.6784387e+00, 1.8850280e+01, 5.6260234e-01,
        5.5715760e-01, 6.7058295

In [13]:
err_threshold = 3 # remove errors that are larger than this...
err_mask_train = np.all(np.abs(full_training_labels_raw) < err_threshold, axis=1)
err_mask_test = np.all(np.abs(full_test_labels_raw) < err_threshold, axis=1)
err_mask_validation = np.all(np.abs(full_validation_labels_raw) < err_threshold, axis=1)

Remove unreasonably huge errors bars or unphysical abundance estimates:

In [14]:
full_training_labels = full_training_labels_raw[err_mask_train]
full_test_labels = full_test_labels_raw[err_mask_test]
full_validation_labels = full_validation_labels_raw[err_mask_validation]

In [15]:
full_training_labels.shape, full_test_labels.shape, full_validation_labels.shape

((114885, 38), (14374, 38), (14326, 38))

In [16]:
np.min(full_training_labels.T, axis=1), np.max(full_training_labels.T, axis=1)

(array([-1.8320073 , -1.2602895 , -2.119044  , -0.18978369, -0.7124527 ,
        -0.1388877 , -0.47890055, -0.13207644, -0.97891414, -0.5242982 ,
        -0.7713039 , -0.40278423, -1.2907453 , -0.6157368 , -1.1658391 ,
        -1.0536897 , -0.64225805, -1.1330373 , -0.2965148 ,  0.01358107,
         0.01762647,  0.02378747,  0.01708818,  0.01984493,  0.02071843,
         0.02232191,  0.01871411,  0.05818468,  0.01897749,  0.03516773,
         0.02518716,  0.02486172,  0.02255343,  0.0375476 ,  0.02254345,
         0.02762588,  0.01909786,  0.02221127]),
 array([0.57460123, 0.81556314, 0.56674075, 1.5901425 , 0.5558584 ,
        0.5102596 , 0.48934165, 0.45210746, 1.0318542 , 0.905827  ,
        1.0242985 , 0.262527  , 0.59501344, 0.8763633 , 0.8669733 ,
        0.27449507, 0.35304812, 0.5658705 , 0.21270613, 0.35667247,
        0.48992783, 1.637775  , 1.0633858 , 0.46304193, 0.43964195,
        0.55135024, 0.46292186, 2.998515  , 0.55373484, 2.87389   ,
        1.0069572 , 0.7626475 , 

In [17]:
from sklearn.preprocessing import StandardScaler

Manual scaler to standardize (equivalent to StandardScaler):

In [18]:
# standardized_pca_array = []
# standardization_means = np.zeros(len(np.array(x_df_transpose)))
# standardization_sigmas = np.zeros(len(np.array(x_df_transpose)))

# for i, param in enumerate(np.array(x_df_transpose)):
#     mean, sigma = np.mean(param), np.std(param)
#     norm = (param - mean)/sigma
#     standardized_pca_array.append(norm)
#     standardization_means[i] = mean
#     standardization_sigmas[i] = sigma

StandardScaler to standardize (same method for t-SNE/UMAP):

In [19]:
from sklearn.preprocessing import StandardScaler

# split abundances and errors into separate datasets
training_labels_abundances = full_training_labels[:, :19]
training_labels_errors = full_training_labels[:, 19:]

test_labels_abundances = full_test_labels[:, :19]
test_labels_errors = full_test_labels[:, 19:]

validation_labels_abundances = full_validation_labels[:, :19]
validation_labels_errors = full_validation_labels[:, 19:]

# standardize means fitting and transforming with standard scaler, standardize errors by scaling by the scale factor (mean will shift but this does not change the errors)
training_scaler = StandardScaler().fit(training_labels_abundances)
standardized_abundance_training_arr = training_scaler.transform(training_labels_abundances)
standardized_error_training_arr = training_labels_errors/training_scaler.scale_
standardized_full_training_arr = np.c_[standardized_abundance_training_arr, standardized_error_training_arr]

test_scaler = StandardScaler().fit(test_labels_abundances)
standardized_abundance_test_arr = test_scaler.transform(test_labels_abundances)
standardized_error_test_arr = test_labels_errors/test_scaler.scale_
standardized_full_test_arr = np.c_[standardized_abundance_test_arr, standardized_error_test_arr]

validation_scaler = StandardScaler().fit(validation_labels_abundances)
standardized_abundance_validation_arr = validation_scaler.transform(validation_labels_abundances)
standardized_error_validation_arr = validation_labels_errors/validation_scaler.scale_
standardized_full_validation_arr = np.c_[standardized_abundance_validation_arr, standardized_error_validation_arr]

Begin VAE stuff:

# SAMPLING LAYER

In [20]:
class Sampling(layers.Layer):
    """Uses (mean, log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        mean, log_var = inputs
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mean + tf.exp(0.5 * log_var) * epsilon

# ENCODER

In [21]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(19,)) #to predict error, inputs is 38

x = tf.keras.layers.Dense(100, activation='gelu')(encoder_inputs)
x = tf.keras.layers.Dense(100, activation='gelu')(x)
x = tf.keras.layers.Dense(100, activation='gelu')(x)
x = tf.keras.layers.Dense(100, activation='gelu')(x)

mean = layers.Dense(latent_dim, name="mean")(x)
log_var = layers.Dense(latent_dim, name="log_var")(x)
z = Sampling()([mean, log_var])
encoder = keras.Model(encoder_inputs, [mean, log_var, z], name="encoder")
encoder.summary()

2025-05-08 09:02:53.178202: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 100)       │      2,000 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 100)       │     10,100 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 100)       │     10,100 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 100)       │     10,100 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mean (Dense)        │ (None, 2)         │        202 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ log_var (Dense)     │ (None, 2)         │        202 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling (Sampling) │ (None, 2)         │          0 │ mean[0][0],       │
│                     │                   │            │ log_var[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 32,704 (127.75 KB)

 Trainable params: 32,704 (127.75 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
#Structure of old NN:
#     tf.keras.layers.Dense(100, activation='gelu'),
#     tf.keras.layers.Dense(100, activation='gelu'),
#     tf.keras.layers.Dense(100, activation='linear'),
#     tf.keras.layers.Dense(38)

# DECODER

In [23]:
latent_inputs = keras.Input(shape=(latent_dim,))

x = tf.keras.layers.Dense(100, activation="gelu")(latent_inputs)
x = tf.keras.layers.Dense(100, activation="gelu")(x)
x = tf.keras.layers.Dense(100, activation="gelu")(x)
x = tf.keras.layers.Dense(100, activation="gelu")(x)

decoder_outputs = layers.Dense(19, activation="linear")(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │           300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 19)             │         1,919 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,519 (127.03 KB)

 Trainable params: 32,519 (127.03 KB)

 Non-trainable params: 0 (0.00 B)

# DEFINE VAE

In [24]:
# alpha = 1e-15  # learning rate
# # optimizer uses Adam with learning rate 1e-7
# optimizer = tf.keras.optimizers.Adam(learning_rate = alpha)

# ### CUSTOM LR SCHEDULER
# lr_start   = 1e-7
# lr_max     = 1e-4
# lr_min     = 1e-12
# lr_ramp_ep  = 3
# lr_sus_ep  = 0
# lr_decay   = 0.7
   
# def lrfn(epoch, lr_start=lr_start, lr_max=lr_max, lr_min=lr_min, lr_ramp_ep=lr_ramp_ep, lr_sus_ep=lr_sus_ep, lr_decay=lr_decay):
#     if epoch < lr_ramp_ep:
#         lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
#     elif epoch < lr_ramp_ep + lr_sus_ep:
#         lr = lr_max
#     else:
#         lr = (lr_max - lr_min) * lr_decay **(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
#     return lr

# # Using this Custom Function, create a Callback
# lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

### COSINE ANNEALING

initial_lr = 1.e-4
decay_steps = int(1e5)  # total number of steps to decay over
alpha = 1.e-16          # minimum learning rate = alpha * initial_lr

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=initial_lr,
    decay_steps=decay_steps,
    alpha=alpha
)

# alpha=1e-7
# optimizer = tf.keras.optimizers.Adam(learning_rate = alpha)
 
# def mse_loss(y_true, y_pred):
#     r_loss = K.mean(K.square(y_true - y_pred), axis = [1,2,3])
#     return 1000 * r_loss
 
# def kl_loss(mean, log_var):
#     kl_loss =  -0.5 * K.sum(1 + log_var - K.square(mean) - K.exp(log_var), axis = 1)
#     return kl_loss
 
# def vae_loss(y_true, y_pred, mean, var):
#     r_loss = mse_loss(y_true, y_pred)
#     kl_loss = kl_loss(mean, log_var)
#     return  r_loss + kl_loss

In [25]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss") 
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.mse_tracker = keras.metrics.Mean(name="mse")
        # self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            # self.kl_loss_tracker,
            self.mse_tracker,
        ]
    # train using WEIGHTED MSE, also record JUST MSE as well
    def train_step(self, args):
        (data, errors), = args
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            # reconstruct data from z
            reconstruction = self.decoder(z)
            # compute the weighted MSE which is the reconstruction loss
            reconstruction_loss = tf.reduce_mean(tf.square(data - reconstruction)/errors) * 1000  # MEAN over all abundances                   
            # compute the MSE loss:
            mse = tf.reduce_mean(tf.square(data - reconstruction)) * 1000  # MEAN over all abundances
            # total loss is JUST the weighted MSE:
            total_loss = reconstruction_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.mse_tracker.update_state(mse)
        # self.kl_loss_tracker.update_state(kl_loss)
        return {
            "total loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            # "kl_loss": self.kl_loss_tracker.result(),
            "mse": self.mse_tracker.result(),
        }
    # evaluate
    def test_step(self, args):
        data, errors = args
        # encode
        z_mean, z_log_var, z = self.encoder(data)
        # reconstruct
        reconstruction = self.decoder(z)
        # compute losses
        # compute the weighted MSE which is the reconstruction loss
        # TEST ERRORS
        reconstruction_loss = tf.reduce_mean(tf.square(data - reconstruction)/errors) * 1000  # MEAN over all abundances                   
        # compute the MSE loss:
        mse = tf.reduce_mean(tf.square(data - reconstruction)) * 1000  # MEAN over all abundances
        # total loss is JUST the weighted MSE:
        total_loss = reconstruction_loss
        # update loss trackers
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.mse_tracker.update_state(mse)
        # return loss
        return {
            "total loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            # "kl_loss": self.kl_loss_tracker.result(),
            "mse": self.mse_tracker.result(),
        }

# TRAIN

Train "naive" VAE:

In [ ]:
# training_labels, error_training_labels = full_training_labels[:, :19], full_training_labels[:, 19:]
# test_labels, error_test_labels = full_test_labels[:, :19], full_test_labels[:, 19:]
# validation_labels, error_validation_labels = full_validation_labels[:, :19], full_validation_labels[:, 19:]

### CHECKPOINTING:

class EpochCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, save_every=50, save_path='naive_vae_checkpoints/ckpt-epoch-{epoch:02d}.weights.h5'):
        super().__init__()
        self.save_every = save_every
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_every == 0:
            filepath = self.save_path.format(epoch=epoch + 1)
            self.model.save_weights(filepath)
            print(f"Checkpoint saved at epoch {epoch + 1}")

# optimizer
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
# VAE
vae = VAE(encoder=encoder, decoder=decoder)
vae.compile(optimizer=optimizer)
# FIT
# vae.fit(training_labels, epochs=5000, batch_size=256, callbacks=lr_callback)
history = vae.fit(x=(standardized_abundance_training_arr, standardized_error_training_arr),
                  epochs=5000, batch_size=256, 
                  validation_data=(standardized_abundance_test_arr, standardized_error_test_arr), 
                  callbacks=[EpochCheckpoint(save_every=50)])

Epoch 1/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - mse: 738.6485 - reconstruction_loss: 1242.7487 - total loss: 1242.7487 - val_total loss: 361.1350 - val_reconstruction_loss: 361.1350 - val_mse: 293.0603
Epoch 2/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - mse: 273.4980 - reconstruction_loss: 308.5888 - total loss: 308.5888 - val_total loss: 302.4166 - val_reconstruction_loss: 302.4166 - val_mse: 258.8140
Epoch 3/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - mse: 247.7616 - reconstruction_loss: 272.8796 - total loss: 272.8796 - val_total loss: 291.7935 - val_reconstruction_loss: 291.7935 - val_mse: 252.5181
Epoch 4/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - mse: 241.0940 - reconstruction_loss: 264.2026 - total loss: 264.2026 - val_total loss: 286.5735 - val_reconstruction_loss: 286.5735 - val_mse: 248.7425
Epoch 5/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - mse: 235.3975 - reconstruction_loss: 258.5454 - total loss: 258.5454 - val_total loss: 283.0872 - val_reconstr

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 170.1306 - reconstruction_loss: 187.6796 - total loss: 187.6796 - val_total loss: 226.4724 - val_reconstruction_loss: 226.4724 - val_mse: 198.0191
Epoch 2093/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 170.6733 - reconstruction_loss: 187.0623 - total loss: 187.0623 - val_total loss: 226.4753 - val_reconstruction_loss: 226.4753 - val_mse: 198.0219
Epoch 2094/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 171.4617 - reconstruction_loss: 187.9451 - total loss: 187.9451 - val_total loss: 226.4657 - val_reconstruction_loss: 226.4657 - val_mse: 198.0156
Epoch 2095/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 170.1750 - reconstruction_loss: 187.0365 - total loss: 187.0365 - val_total loss: 226.4763 - val_reconstruction_loss: 226.4763 - val_mse: 198.0182
Epoch 2096/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 170.4350 - reconstruction_loss: 187.2047 - total loss: 187.2047 - val_total loss: 226.4660 - val_reconstruct

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - mse: 170.9842 - reconstruction_loss: 187.2083 - total loss: 187.2083 - val_total loss: 226.4750 - val_reconstruction_loss: 226.4750 - val_mse: 198.0183
Epoch 2343/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 168.6852 - reconstruction_loss: 186.6952 - total loss: 186.6952 - val_total loss: 226.4691 - val_reconstruction_loss: 226.4691 - val_mse: 198.0165
Epoch 2344/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 171.3102 - reconstruction_loss: 186.6965 - total loss: 186.6965 - val_total loss: 226.4725 - val_reconstruction_loss: 226.4725 - val_mse: 198.0181
Epoch 2345/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 171.0469 - reconstruction_loss: 187.7678 - total loss: 187.7678 - val_total loss: 226.4711 - val_reconstruction_loss: 226.4711 - val_mse: 198.0196
Epoch 2346/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 171.1395 - reconstruction_loss: 187.0245 - total loss: 187.0245 - val_total loss: 226.4784 - val_reconstruct

...

449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - mse: 170.9849 - reconstruction_loss: 187.2332 - total loss: 187.2332 - val_total loss: 226.4747 - val_reconstruction_loss: 226.4747 - val_mse: 198.0182
Epoch 3537/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 170.3809 - reconstruction_loss: 187.4500 - total loss: 187.4500 - val_total loss: 226.4810 - val_reconstruction_loss: 226.4810 - val_mse: 198.0223
Epoch 3538/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - mse: 171.5450 - reconstruction_loss: 186.8800 - total loss: 186.8800 - val_total loss: 226.4778 - val_reconstruction_loss: 226.4778 - val_mse: 198.0219
Epoch 3539/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - mse: 169.9177 - reconstruction_loss: 187.4155 - total loss: 187.4155 - val_total loss: 226.4664 - val_reconstruction_loss: 226.4664 - val_mse: 198.0172
Epoch 3540/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 169.6918 - reconstruction_loss: 186.4703 - total loss: 186.4703 - val_total loss: 226.4753 - val_reconstruct

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 172.1961 - reconstruction_loss: 187.4573 - total loss: 187.4573 - val_total loss: 226.4790 - val_reconstruction_loss: 226.4790 - val_mse: 198.0199
Epoch 3726/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 169.8434 - reconstruction_loss: 186.8379 - total loss: 186.8379 - val_total loss: 226.4632 - val_reconstruction_loss: 226.4632 - val_mse: 198.0149
Epoch 3727/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 170.7597 - reconstruction_loss: 186.2877 - total loss: 186.2877 - val_total loss: 226.4762 - val_reconstruction_loss: 226.4762 - val_mse: 198.0178
Epoch 3728/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 170.4248 - reconstruction_loss: 186.5969 - total loss: 186.5969 - val_total loss: 226.4720 - val_reconstruction_loss: 226.4720 - val_mse: 198.0191
Epoch 3729/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - mse: 170.1962 - reconstruction_loss: 187.2994 - total loss: 187.2994 - val_total loss: 226.4685 - val_reconstruct

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - mse: 169.8056 - reconstruction_loss: 186.7425 - total loss: 186.7425 - val_total loss: 226.4695 - val_reconstruction_loss: 226.4695 - val_mse: 198.0165
Epoch 3914/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - mse: 170.8345 - reconstruction_loss: 187.0947 - total loss: 187.0947 - val_total loss: 226.4743 - val_reconstruction_loss: 226.4743 - val_mse: 198.0182
Epoch 3915/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - mse: 170.3726 - reconstruction_loss: 187.1093 - total loss: 187.1093 - val_total loss: 226.4751 - val_reconstruction_loss: 226.4751 - val_mse: 198.0199
Epoch 3916/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - mse: 170.0612 - reconstruction_loss: 187.0511 - total loss: 187.0511 - val_total loss: 226.4717 - val_reconstruction_loss: 226.4717 - val_mse: 198.0183
Epoch 3917/5000
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - mse: 170.6358 - reconstruction_loss: 187.3904 - total loss: 187.3904 - val_total loss: 226.4738 - val_reconstruct

save:

In [ ]:
# save the model?
encoder.save('encoder_naive_vae.keras', overwrite=True)#, save_format='tf')
decoder.save('decoder_naive_vae.keras', overwrite=True)#, save_format='tf')
# save the history as well
# Save history object
with open('naive_vae_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

Load encoder, decoder and history:

In [ ]:
keras.utils.get_custom_objects().update({
    "Sampling": Sampling,
})

encoder = tf.keras.models.load_model('encoder_naive_vae.keras')
decoder = tf.keras.models.load_model('decoder_naive_vae.keras')

with open('naive_vae_history.pkl', 'rb') as f:
    history = pickle.load(f)

len(history['total loss'])

In [ ]:
# plt.ylim(4.5, 7.)
# plt.plot(history['reconstruction_loss'])
# plt.plot(history['kl loss'])
plt.plot(history['total loss'], linewidth=2.5, color='dodgerblue', label = 'Training')
plt.plot(history['val_total loss'], linewidth=2.5, color='darkorange', label = 'Test')
plt.xlim(0, 300)
plt.xlabel('Epoch'), plt.ylabel('Loss')
plt.legend()

In [ ]:
testmean, testvar, testz = encoder(standardized_abundance_test_arr)
test_reconstruction = decoder(testz)
standardized_abundance_test_arr, test_reconstruction

In [ ]:
# mses = (test_reconstruction - standardized_abundance_test_arr)[:,0]

plt.figure(figsize = (12, 6))
plt.scatter(standardized_abundance_test_arr[:,0], test_reconstruction[:,0], s=0.2)
# plt.ylim(bottom=0., top=0.1)

In [ ]:
ind = 5  # 19 parameters, 0 to 18
samp_rate = 1

plt.figure(figsize = (12, 6))
plt.scatter(standardized_abundance_test_arr[:,ind][::samp_rate], test_reconstruction[:,ind][::samp_rate], s=0.1)#, xerr=np.exp(standardized_abundance_test_arr[:,ind+19][::samp_rate]/2), yerr=np.exp(test_reconstruction[:,ind+19][::samp_rate]/2), fmt='.', markersize=5, elinewidth=0.5)
# plt.plot(np.linspace(0., 0.3, int(1e4)), np.linspace(0., 0.3, int(1e4)), c='r')
plt.plot(np.linspace(np.min(standardized_abundance_test_arr[:,ind]), np.max(standardized_abundance_test_arr[:,ind]), int(1e5)), np.linspace(np.min(standardized_abundance_test_arr[:,ind]), np.max(standardized_abundance_test_arr[:,ind]), int(1e5)), c='r')
plt.xlabel('Original'), plt.ylabel('Reconstructed')
# plt.xlim(-0.5, 0.4)
# plt.ylim(-0.5, 0.4)

In [ ]:
import matplotlib as mpl
plt.rc('font', size=18)

ind=0

fig = plt.figure(figsize = (14, 8))
# ax = fig.add_subplot(111)    # The big subplot
ax0 = fig.add_subplot(221)
ax1 = fig.add_subplot(222)
# ax2 = fig.add_subplot(223)
# ax3 = fig.add_subplot(224)
# plot the metallicity
ax0.scatter(standardized_abundance_test_arr[:,ind], test_reconstruction[:,ind], s=0.1)
ax0.plot(np.linspace(np.min(standardized_abundance_test_arr[:,ind]), np.max(standardized_abundance_test_arr[:,ind]), int(1e5)), np.linspace(np.min(standardized_abundance_test_arr[:,ind]), np.max(standardized_abundance_test_arr[:,ind]), int(1e5)), c='r')
ax0.set_xlabel('Fe/H'), ax0.set_ylabel('Reconstructed')
# plot the residuals of metallicity
# ax1.hist2d(standardized_abundance_test_arr[:,ind], standardized_abundance_test_arr[:,ind] - test_reconstruction[:,ind], norm=mpl.colors.LogNorm(), bins=200)
ax1.scatter(standardized_abundance_test_arr[:,ind], standardized_abundance_test_arr[:,ind] - test_reconstruction[:,ind], s=0.1)
ax1.set_xlabel(r'Fe/H'), ax1.set_ylabel('Residual')
# ax1.scatter(standardized_abundance_test_arr[:,ind], standardized_abundance_test_arr[:,ind] - test_reconstruction[:,ind], s=0.1)
ax1.axhline(0, c='r')
plt.tight_layout()
plt.savefig('metallicity_original_vs_reconstructed_naive.png')

In [ ]:
np.std(standardized_abundance_test_arr[:,ind] - test_reconstruction[:,ind])

Plot:

In [ ]:
abundances = ['Fe/H', 'C/Fe', 'Ci/Fe', 'N/Fe', 'O/Fe', 'Mg/Fe', 'Al/Fe', 'Si/Fe', 'P/Fe', 'S/Fe', 'K/Fe', 'Ca/Fe', 'Ti/Fe', 'Tiii/Fe', 'V/Fe', 'Cr/Fe', 'Mn/Fe', 'Co/Fe', 'Ni/Fe']
divisor = 4  # fraction of the range to scale the plots by

plt.rc('font', size=14)
fig, ax = plt.subplots(6, 3, figsize=(14, 14))
for i in range(6):
    for j in range(3):
        ax[i, j].scatter(standardized_abundance_test_arr[:,3*i+j+1][::samp_rate], test_reconstruction[:,3*i+j+1][::samp_rate], s=0.1)
        # plt.plot(np.linspace(0., 0.3, int(1e4)), np.linspace(0., 0.3, int(1e4)), c='r')
        ax[i, j].plot(np.linspace(np.min(standardized_abundance_test_arr[:,3*i+j+1]), np.max(standardized_abundance_test_arr[:,3*i+j+1]), int(1e5)), 
                      np.linspace(np.min(standardized_abundance_test_arr[:,3*i+j+1]), np.max(standardized_abundance_test_arr[:,3*i+j+1]), int(1e5)), c='r')
        ax[i, j].set_xlabel(abundances[3*i+j+1])# , ax[i, j].set_ylabel('Reconstructed')       
        # scale it properly based on reconstruction
        abundance_min, abundance_max = np.quantile(standardized_abundance_test_arr[:,3*i+j+1], (0.005, 0.995))
        # abundance_min = np.min(test_reconstruction[:, 3*i + j])
        # abundance_max = np.max(test_reconstruction[:, 3*i + j])
        diff = np.abs(abundance_max - abundance_min)
        # scale it
        ax[i, j].set_xlim(abundance_min - diff/divisor, abundance_max + diff/divisor) 
        ax[i, j].set_ylim(abundance_min - diff/divisor, abundance_max + diff/divisor) 
        
plt.tight_layout()
plt.savefig('original_vs_reconstructed_naive_all.png')# , fmt='png')
plt.show()

In [ ]:
plt.scatter(standardized_abundance_test_arr[:,17], (standardized_abundance_test_arr[:,17] - test_reconstruction[:, 17]), s=0.1)